# Programmieraufgabe: CG-Verfahren und PCG-Verfahren

<span style="color:red">Die bisherige Version von Programmieraufgabe 6 war leider missverständlich gestellt. Sie dürfen gerne diese aktualisierte Version benutzen.</span>

<span style="color:red">Änderungen:</span>
* <span style="color:red">Sie dürfen eine vereinfachte Variante der Matrix $A$ benutzen.</span>
* <span style="color:red">Die Methoden `cg` und `pcg` wurden an das Skript angepasst.</span>

Wie in Beispiel 3.39 im Skript betrachten wir ein quadratisches Optimierungsproblem, das aus der zweidimensionalen Wärmeleitungsgleichung resultiert. Dabei hat die Matrix $A$ die Form
$$
 A = \begin{bmatrix}
    B  & C     &        &    \\
    C^T & \ddots & \ddots &    \\
       & \ddots & \ddots & C \\
       &        &     C^T & B
 \end{bmatrix} \in \mathbb{R}^{n \times n}
 $$
 mit 
 $$
 B = \begin{bmatrix}
    4  & -1     &        &    \\
    -1 & \ddots & \ddots &    \\
       & \ddots & \ddots & -1 \\
       &        &     -1 & 4
    \end{bmatrix} \in \mathbb{R}^{\sqrt{n}\times \sqrt{n}}
$$
und
$$
C = \begin{bmatrix}
    -1  & 0      &        &   \\
    0   & \ddots & \ddots &   \\
        & \ddots & \ddots &  0\\
    -1  &        &      0 & -1
    \end{bmatrix} \in \mathbb{R}^{\sqrt{n}\times \sqrt{n}}.
$$

$A$ ist also eine symmetrische Matrix mit $4$ auf der Diagonalen und $-1$ auf den ersten und $\sqrt{n}$-ten Nebendiagonalen.

Ziel ist es, dieses System mithilfe des **CG-Verfahrens** (konjugierte Gradienten) und des **PCG-Verfahrens** (präkonditionierte Gradienten) zu lösen.

Tragen Sie zunächst in der folgenden Zelle **beide** Ihre Namen ein: 

In [ ]:
# Numerische Optimierungsverfahren der Wirtschaftsmathematik
# Wintersemester 2025/2026
# Übungsblatt 6 - Programmieraufgabe 6
#
# [Nachname], [Vorname]
# [Vorname.Nachname@uni-a.de]
# 
# [Nachname], [Vorname]
# [Vorname.Nachname@uni-a.de]

In [ ]:
import math
import time
import numpy as np
from numpy.linalg import norm
from numpy.linalg import inv
import matplotlib.pyplot as plt
import sys

## Teil 1: Implementierung der Hilfsfunktionen

Für großes $n$ ist es sehr ineffizient, die Matrix $A$ und den Präkonditionierer $S$ zu speichern. Daher wollen wir die Anwendung dieser Matrizen direkt als Funktionen implementieren. Dieser Ansatz wird auch als "Matrix-Free" bezeichnet.

* `Av = mul_A(v)`: Diese Funktion soll die Multiplikation der Matrix $A$ mit einem beliebigen Vektor $v$ effizient realisieren, **ohne** dass $A$ als $n \times n$-Matrix gespeichert werden muss.
* `w  = mul_pre(r)`: Analog soll diese Funktion die Multiplikation eines Vektors $r$ mit $S^{-T} S^{-1}$ realisieren.

Für die Präkonditionierung soll, wie in der Vorlesung besprochen, $S = \frac{1}{2} D + L^T$ verwendet werden, wobei die Zerlegung $A = L + D + L^T$ zugrunde liegt ($L$ ist der strikte obere Dreiecksteil und $D$ die Diagonale von $A$). 

$S^{-T}S^{-1}$ kann dann leicht mittel Vorwärts- und Rückwärtssubstitution angewendet werden.

In [ ]:
def mul_A(v):
    """
    A*v = mul_A(v)
    Legen Sie hier nicht die Matrix A an!
    """
    sqrt_n = int(np.sqrt(v.shape[0]))
    Av = 4.0 * v
    Av[:-1] -= ???
    Av[1:] -= ???
    Av[:-sqrt_n] -= ???
    Av[sqrt_n:] -= ???
    return Av

In [ ]:
def mul_pre(r):
    """
    Anwendung des Vorkonditionierers S^(-1)S^(-T) auf r
    (Auch hier dürfen weder A noch S^(-1) als Matrix angelegt werden)
    """
    n = r.shape[0]
    sqrt_n = int(np.sqrt(n))
    # z = S^(-T)*r
    z = np.zeros(n)
    z[0] = r[0] / 2.0
    for i in range(1, n):
        z[i] = (r[i] + z[i-1]) / 2.0
        if i >= int(sqrt_n):
            z[i] += z[i - sqrt_n] / 2.0
    # w = S^(-1)*z
    w = np.zeros(n)
        ???
    return w

Sie können die folgenden beiden Zeilen zum Testen Ihrer Funktionen verwenden:

In [ ]:
m = 4
n = 16
A = 4.0 * np.eye(n) - (np.diag(np.ones(n-1), k=-1) + np.diag(np.ones(n-1), k=1))
A -= (np.diag(np.ones(n - m), k = m) + np.diag(np.ones(n - m), k = -m))
x = np.ones(n)
print(A @ x)
print(mul_A(x))
assert (A @ x == mul_A(x)).all()

In [ ]:
m = 4
n = 16
S =  2.0 * np.eye(n) - np.diag(np.ones(n-1), k=1)
S -= np.diag(np.ones(n - m), k = m)
b = np.ones(n)
print(inv(S) @ inv(S.T) @ b)
print(mul_pre(b))
assert (inv(S) @ inv(S.T) @ b == mul_pre(b)).all()

### Teil 2: Implementierung der CG-Verfahren

Als Nächstes werden die Hauptfunktionen für die iterativen Lösungsverfahren benötigt:

* `iterates, gradient_norms = cg(b, mul_A, tol, kmax)`
* `iterates, gradient_norms = pcg(b, mul_A, mul_pre, tol, kmax)`

Diesen Funktionen werden `mul_A` und `mul_pre` als Parameter übergeben, um die Matrixoperationen durchzuführen.

Parameter:
* `tol` ist die Fehlertoleranz. Die Verfahren sollen abgebrochen werden, wenn **sowohl** die Norm der Gradienten **als auch** die Norm der Differenz aufeinanderfolgender Iterierter $\leq$ `tol` ist.
* `kmax` ist die maximale Anzahl an Iterationen. Die Verfahren müssen spätestens terminieren, wenn diese Grenze erreicht ist.
* Als Startvektor wählen Sie die rechte Seite $b$.


Die Rückgabewerte der Funktionen `cg` und `pcg` sollen dabei sein:
* `iterates` (Vektor): Die vom jeweiligen Verfahren berechneten Iterierten, wobei `iterates[-1,:]` die berechnete Lösung ist.
* `gradient_norms` (Vektor): Der Vektor mit den Normen der Gradienten $\|Ax^{(i)} + b\|_2$.


In [ ]:
def cg(b, mul_A, tol, max_iter):
    """
    Konjugierte-Gradienten-Verfahren (CG)
        iterates, gradient_norms = cg(b, mul_A, tol, max_iter)

    Parameter:
      b        : Rechte Seite und Initialschätzung (Startvektor x0);
      mul_A    : Funktion, die A*v berechnet;
      tol      : Geforderte absolute Genauigkeit;
      max_iter : Maximale Zahl von Iterationen.

    Resultate:
      iterates       : Folge der Iterierten x^(k)
      gradient_norms : Folge der Gradientennormen ||A*x^(i) + b||_2;

    Das Verfahren bricht ab, wenn entweder die maximale Zahl von
    Iterationsschritten 'kmax' erreicht ist oder die 2-Normen des
    Gradienten (A*x + b) und Schrittweite (x^(i+1) - x^(i) = alpha^(i)*v^(i))
    beide kleiner-gleich der Toleranz 'tol' sind.
    """

    n = b.shape[0]
    gradient_norms = np.zeros(max_iter + 1)   # Gradientennorm || A*x^(i) + b ||_2

    # Initialisierung
    iter_count = 0                            # Zählt die berechneten Iterationsschritte
    iterates = np.zeros((max_iter + 1, n))
    iterates[0,:] = np.copy(b)                # x^(0) = b
    gradient = mul_A(iterates[0,:]) + b       # g^(0) = A*x^(0) + b
    direction = -1 * np.copy(gradient)        # d^(0) = -g^(0)
    gamma_old = np.inner(gradient, gradient)  # gamma^(0) = <g^(0), g^(0)>
    step_norm = np.inf                        # || alpha^(i) * v^(i) ||

    gradient_norms[iter_count] = norm(gradient)

    # Iteration
    ???

    # Die resultierenden Vektoren sollen genau die richtige Länge haben;
    # alle überschüssigen Elemente werden abgeschnitten:
    gradient_norms = gradient_norms[:iter_count + 1]
    iterates = iterates[:iter_count + 1,:]

    return iterates, gradient_norms

In [ ]:
def pcg(b, mul_A, mul_pre, tol, max_iter):
    """
    Präkonditioniertes Konjugierte-Gradienten-Verfahren (PCG)
        iterates, gradient_norms = pcg(b, mul_A, mul_pre, tol, max_iter)

    Parameter:
      b        : Rechte Seite und Initialschätzung (Startvektor x0);
      mul_A    : Funktion, die A*v berechnet;
      mul_pre  : Funktion, die M^(-1)*v berechnet (M^(-1) = S^(-T)*S^(-1));
      tol      : Geforderte absolute Genauigkeit;
      max_iter : Maximale Zahl von Iterationen.

    Resultate:
      iterates  : Folge der Iterierten x^(k)
      gradient_norms : Folge der Gradientennormen ||A*x^(i) + b||_2;

    Das Verfahren bricht ab, wenn entweder die maximale Zahl von
    Iterationsschritten 'kmax' erreicht ist oder die 2-Normen des
    Gradienten (A*x + b) und Schrittweite (x^(i+1) - x^(i) = alpha^(i)*v^(i))
    beide kleiner-gleich der Toleranz 'tol' sind.
    """
    
    n = b.shape[0]
    gradient_norms = np.zeros(max_iter + 1)               # Gradientennorm || A*x^(i) + b ||_2

    # Initialisierung
    iter_count = 0                                        # Zählt die berechneten Iterationsschritte
    iterates = np.zeros((max_iter + 1, n))
    iterates[0,:] = np.copy(b)                            # x^(0) = b
    gradient = mul_A(iterates[0,:]) + b                   # g^(0) = A*x^(0) + b
    preconditioned_gradient =  mul_pre(gradient)          # w_0 = M^(-1) * g_0
    direction = -1 * np.copy(preconditioned_gradient)     # p_0 = -w_0
    gamma_old = np.dot(preconditioned_gradient, gradient) # gamma_0 = <w_0, g_0>
    step_norm = np.inf                                    # || alpha^(i) * v^(i) ||

    gradient_norms[iter_count] = norm(gradient)

    # Iteration
    ???

    # Die resultierenden Vektoren sollen genau die richtige Länge haben;
    # alle überschuessigen Elemente werden daher abgeschnitten:
    gradient_norms = gradient_norms[:iter_count + 1]
    iterates = iterates[:iter_count + 1,:]

    return iterates, gradient_norms

Sie können die folgenden beiden Zeilen zum Testen Ihrer Funktionen verwenden. Dabei sollte `iterates.shape` die Form `(anzahl_iterationen, 25)` und `gradient_norms.shape` die Form `(anzahl_iterationen,)` haben.

In [ ]:
iterates, gradient_norms = cg(np.ones(5*5), mul_A, 1e-5, 20)
print(iterates.shape)
print(gradient_norms.shape)

In [ ]:
iterates, gradient_norms = pcg(np.ones(5*5), mul_A, mul_pre, 1e-5, 20)
print(iterates.shape)
print(gradient_norms.shape)

### Teil 3: Vergleich der Verfahren

Nun wollen wir die implementierten Verfahren vergleichen. Dazu wenden wir die Funktionen `cg` und `pcg` auf eine Matrix $A \in \mathbb{R}^{n \times n}$ an, wobei $n = m^2$ für ein $m \in \mathbb{N}$.

In [ ]:
from util.plotting_06 import compute_and_plot_06

maxit = 5000
tol = 1e-15
m = 50

b, cg_solution, pcg_solution = compute_and_plot_06(cg, pcg, m, mul_A, mul_pre, tol, maxit)

Zum Schluss können wir auch noch unsere Lösung der Wärmeleitgleichung plotten:

In [ ]:
plt.figure()
plt.subplot(121)
plt.imshow(b.reshape(m,m))
plt.title("$b$")
plt.subplot(122)
plt.imshow(-1 * pcg_solution.reshape(m, m)) # pcg_solution ist argmax 0.5*x^TAx + b^Tx = -A^(-1)b
plt.title("$A^{-1}b$") 
plt.show()

Optionale Zusatzfrage (ohne Bewertung): Wie würde es sich optisch auf das Ergebnis $A^{-1}b$ auswirken, wenn Sie die Matrix $A$ wie in Beispiel 3.39 im Skript, also mit $C = -I$ wählen würden?